Baseline models
* Regression
* CLassification
    * BlockStackNet4() googlenet-(64,6) - highest validation score 0.4863

In [1]:
import torch.nn as nn
from torchvision import models

## Baseline Models is defined
gogglenet





## The data set file path is defined here.

In [3]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

'''
In this function block, the funciton is defined to do
1. def__init__() : get the image from the trainning data path
2. def __len__() : get the size of hte dataset
3. def __getitem__() : get the ralated image the correct dataset
'''
class BlockStackDataset(Dataset):
  '''
  Return the image and its label. If not access or not exist, raise error.
  '''
  def __init__(self, data_frame, img_dir, transform = None):
    self.data_frame = data_frame
    self.img_dir = img_dir
    self.transform = transform

    if not os.path.exists(self.img_dir):
      raise ValueError(f"File path {self.img_dir} not exisits!")
    if not os.access(self.img_dir, os.R_OK):
      raise ValueError(f"File path {self.img_dir} is not readable!")
  '''
  Return the size of the dataset
  '''
  def __len__(self):
    return len(self.data_frame)

  '''
  Read the image and according to its id, get the related data in the train.csv.

  Then return the corresponding image , stable_height, instability_type dataset provided.
  '''
  def __getitem__(self, idx):
    img_name = os.path.join(self.img_dir, str(self.data_frame.iloc[idx, 0])) # the first column in the train == image name
    image = Image.open(img_name + ".jpg")
    label = self.data_frame.iloc[idx,3] # stable_height/instability_type
    # label = label + 1
    instability_type = self.data_frame.iloc[idx,4] # instability_type

    if self.transform:
      image = self.transform(image)

    return image, label, instability_type



In [4]:
df = pd.read_csv('./COMP90086_2024_Project_train/train.csv')
df.head()

,id,shapeset,type,total_height,instability_type,cam_angle,stable_height
0,54,2,1,3,1,1,2
1,173,1,1,4,1,2,1
2,245,1,1,4,1,2,1
3,465,2,1,5,0,1,5
4,611,2,1,3,1,1,1


## Trainner

In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix,classification_report
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm import tqdm
from datetime import datetime as datatime



In [6]:
'''
The function here helps to create the logs recording the experiment results.
'''
def create_log_dir():
  current_time = datatime.now().strftime("%Y%m%d-%H%M%S")
  # log_dir = f'/content/drive/MyDrive/CV final project/runs/experiment_{current_time}'
  # solution_dir = f'/content/drive/MyDrive/CV final project/trained_models/experiment_{current_time}'

  log_dir = f'./runs/experiment_{current_time}'
  solution_dir = f'./runs/experiment_{current_time}'
  if not os.path.exists(log_dir):
    os.makedirs(log_dir)

  if not os.path.exists(solution_dir):
    os.makedirs(solution_dir)

  return log_dir, solution_dir


In [84]:

googlenet = models.googlenet(weights=models.GoogLeNet_Weights.IMAGENET1K_V1)
googlenet.fc = nn.Identity()
googlenet.dropout = nn.Identity()
googlenet.avgpool = nn.Identity()
googlenet.inception5b = nn.Identity()
googlenet.inception5a = nn.Identity()
googlenet.maxpool4 = nn.Identity()
# 打印模型结构，找到最后一个卷积层
print(googlenet)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [85]:
import torch

# 创建一个随机的输入 (batch_size=1, channels=3, height=224, width=224)
inputs = torch.randn(1, 3, 224, 224)

# 获取 GoogeLeNet 的特征图
with torch.no_grad():
    output = googlenet(inputs)

# 打印输出的形状
print(output.shape)

torch.Size([1, 163072])


In [89]:

# # Define models: googlenet

# class TunnedBlockStackNet8(nn.Module):
#     def __init__(self):
#         super(TunnedBlockStackNet8, self).__init__()
#         # load the pre-trained model: gogglenet
#         self.googlenet = models.googlenet(weights = models.GoogLeNet_Weights.IMAGENET1K_V1)

#        #  num_ftrs = self.googlenet.fc.in_features
#         self.googlenet.fc = nn.Identity()
#         self.googlenet.dropout = nn.Identity()
#         self.googlenet.avgpool = nn.Identity()
#         self.googlenet.inception5b = nn.Identity()
#         self.googlenet.inception5a = nn.Identity()
#         self.googlenet.maxpool4 = nn.Identity()

#         self.additional_conv_layers = nn.Sequential(
#             # 第一卷积层: 输入通道1024，输出通道512，卷积核3x3，步幅1，padding 1
#             nn.Conv2d(in_channels=832, out_channels=512, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm2d(512),  # 添加批量归一化
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),  # 最大池化层，输出特征图减半

#             # 第二卷积层: 输入通道512，输出通道256，卷积核3x3，步幅1，padding 1
#             nn.Conv2d(in_channels=512, out_channels=256, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm2d(256),  # 添加批量归一化
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2)  # 最大池化层，输出特征图再减半
#         )

#         self.fc = nn.Sequential(
#             # nn.Linear(256, 256),
#             # nn.ReLU(),
#             # nn.Dropout(0.25),
#             nn.Linear(256, 128),
#             nn.ReLU(),
#             nn.Dropout(0.25),
#             nn.Linear(128, 6)

#         )
#     def forward(self, x):
#         x = self.googlenet(x)
#         # 知道通道数 channels，假设这里为 1024
#         channels = 832

#         # 动态计算 height 和 width
#         total_size = x.size(1)  # 获取展平后的总大小
#         height_width = int((total_size // channels) ** 0.5)  # 计算 height 和 width

#         # 将张量转换为 4D 张量 [batch_size, channels, height, width]
#         x = x.view(x.size(0), channels, height_width, height_width)

#      #   x = x.view(32,1024,7,7)
#        # x = x.view(x.size(0), -1)  # 展平成 (batch_size, 1024 * 7 * 7)

#         # 经过附加的卷积层
#         x = self.additional_conv_layers(x)  # 输出尺寸为 (batch_size, 256, 7, 7)
#         # 平铺卷积层输出，以输入到全连接层
#         x = x.view(x.size(0), -1)  # 
#         print(x.shape)
#         x = self.fc(x)
#         return x

In [7]:

# Define models: googlenet

class TunnedBlockStackNet8(nn.Module):
    def __init__(self):
        super(TunnedBlockStackNet8, self).__init__()
        # load the pre-trained model: gogglenet
        self.googlenet = models.googlenet(weights = models.GoogLeNet_Weights.IMAGENET1K_V1)

        num_ftrs = self.googlenet.fc.in_features
        self.googlenet.fc = nn.Identity()

        self.fc = nn.Sequential(
            nn.Linear(num_ftrs, 256),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(128, 6)

        )
    def forward(self, x):
      x = self.googlenet(x)
      x = self.fc(x)
      return x

In [9]:
class ClassificationBlockStackTrainer1:
  '''
  The function here helps to initalize the parameters used in the models and pre-process the image
  '''
  def __init__(self, csv_file, img_dir, model, stratify_column='stable_height', test_size=0.2,
                 batch_size=32, num_epochs=10, learning_rate=0.001 ,random_state=42):
        self.csv_file = csv_file
        self.img_dir = img_dir
        self.stratify_column = stratify_column
        self.test_size = test_size
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.model = model

        # load the train dataset
        self.data_frame = pd.read_csv(csv_file)

        # split data into train and validation dataset
        self.train_data, self.val_data, self.train_ids, self.valid_ids = self.split_dataset()

        # pre-processing the images
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),

              #ADD MORE TRANSFORM METHODS HERE

            transforms.RandomHorizontalFlip(),## ADDDED
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)), ## ADDDED
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),## ADDDED
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])


        self.train_loader = self.create_dataloader(self.train_data, self.transform)
        self.val_loader = self.create_dataloader(self.val_data, self.transform, shuffle=False)


        # use the gpu
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # set the loss weight
        '''
        Changed some thing here : to see the results
        1. add the class_weights to each class labels with different weights,
           more lables occurece with lower weight, few label occurence with higher weight;
           without the weights, all the labels will be treated equally.

        2. used to apply L2 regularization (also called weight decay).
        The primary purpose of weight decay is to prevent overfitting by penalizing large weights.
         It adds a penalty to the loss function based on the size of the weights
         helps it generalize better to unseen data.
         Lnew = Lold + weight_decay * sum(weight^2)

        3. add scheduler to Reduces the learning rate after every step_size epochs.
          After every 4 epochs, the learning rate will be multiplied by gamma (0.1 here), reducing it by 90%.
        '''
        class_weights = torch.tensor([100/25 , 100/25, 100/20,  100/15, 100/10,  100/5], device= self.device)
        self.criterion = nn.CrossEntropyLoss(weight=class_weights) # CrossEntropy for multi categorical-label predication
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate , weight_decay=0.0001)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size = 15, gamma=0.1)


  '''
  The function helps to split the data set into the training and validation dataset according to the
  size pre-determined.
  '''

  def split_dataset(self):
    split = StratifiedShuffleSplit(n_splits=1, test_size=self.test_size, random_state=42)
    train_ids = []
    valid_ids = []
    for train_idx, val_idx in split.split(self.data_frame, self.data_frame[self.stratify_column]):
      train_data = self.data_frame.iloc[train_idx]
      val_data = self.data_frame.iloc[val_idx]
      train_ids.append(train_idx)
      valid_ids.append(val_idx)
    print(f"Train dataset size: {len(train_data)}",
       f"Validation dataset size: {len(val_data)}",
       f"length of train_ids{(len(train_ids))}",
       f"length of valid_ids{(len(valid_ids))}")
    return train_data, val_data, train_ids, valid_ids


  '''
  The function helps to loda the image
  '''
  def create_dataloader(self, data_frame, transform, shuffle=True):
    dataset = BlockStackDataset(data_frame, self.img_dir, transform=transform) # transform 可以用来数据增强
    return DataLoader(dataset, batch_size=self.batch_size, shuffle=shuffle)


  def generate_classification_report(self, outputs, labels):
    predicted = outputs # 'outputs' is already a numpy array after prediction
    labels = labels
    print(classification_report(labels, predicted, zero_division=0))



  def calculate_confusion_matrix(self, outputs, labels):
    predicted = outputs
    labels = labels
    matrix = confusion_matrix(labels, predicted)
    print(matrix)


  def validate(self):
    self.model.eval()
    val_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    all_labels = []  # add all the lables
    all_predictions = []  # add all the prediction

    with torch.no_grad():
        for inputs, labels, _ in self.val_loader:
            inputs, labels = inputs.to(self.device), labels.to(self.device).long()
            labels = labels - 1
            outputs = self.model(inputs)
            predicted = torch.argmax(outputs, 1)

            # collecting all the lables and predictions
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

            # calculate the loss
            loss = self.criterion(outputs, labels)
            val_loss += loss.item()

            # calculate the correct predication
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    # calculate the accuracy rate
    val_accuracy = correct_predictions / total_samples
    self.generate_classification_report(np.array(all_predictions), np.array(all_labels))
    self.calculate_confusion_matrix(np.array(all_predictions), np.array(all_labels))
    return val_loss/len(self.val_loader),val_accuracy, all_labels, all_predictions


  '''
  The function here is used as the main training function on the image by using the pre-definned models in
  hte first model class.
  '''

  def train(self):
      _, solution_dir = create_log_dir()

      best_val_accuracy = 0.0
      for epoch in range(self.num_epochs):
          self.model.train()
          running_loss = 0.0
          running_accuracy = 0.0

          # monitor the process
          with tqdm(self.train_loader, unit="batch") as tepoch:
              tepoch.set_description(f"Epoch {epoch + 1}/{self.num_epochs}")

              for inputs, labels, _ in tepoch:
                  inputs, labels = inputs.to(self.device), labels.to(self.device).long()
                  labels = labels - 1

                  #forward propagation
                  self.optimizer.zero_grad()
                  raw_outputs = self.model(inputs)
                  loss = self.criterion(raw_outputs, labels)  # loss calculation

                  loss.backward()  # backward propagation
                  self.optimizer.step()

                  #Loss calculating
                  running_loss += loss.item()
                  _, predicted = torch.max(raw_outputs, 1)
                  accuracy = (predicted == labels).sum().item()/ labels.size(0)
                  running_accuracy += accuracy
                  tepoch.set_postfix(loss=running_loss / len(self.train_loader),
                            accuracy=running_accuracy / len(self.train_loader))



          self.scheduler.step()
          print(self.scheduler.get_last_lr())


          val_loss,val_accuracy, all_labels, all_predictions = self.validate()
          print(f"Epoch {epoch + 1}/{self.num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

          if val_accuracy > best_val_accuracy:
              best_val_accuracy = val_accuracy
              torch.save(self.model.state_dict(), f'{solution_dir}/best_model.pth')
              current_time = datatime.now().strftime("%Y%m%d-%H%M%S")
              print('Best model saved!',current_time)

      print('Finished Training')
      print(f'Best validation accuracy: {best_val_accuracy:.4f}')



In [95]:
class ClassificationBlockStackTrainer2:
  '''
  The function here helps to initalize the parameters used in the models and pre-process the image
  '''
  def __init__(self, csv_file, img_dir, model, stratify_column='stable_height', test_size=0.2,
                 batch_size=32, num_epochs=10, learning_rate=0.01 ,random_state=42):
        self.csv_file = csv_file
        self.img_dir = img_dir
        self.stratify_column = stratify_column
        self.test_size = test_size
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.model = model

        # load the train dataset
        data_frame = pd.read_csv(csv_file)

        data_frame1 = data_frame[data_frame['instability_type']==2]
        randome_instable2 = data_frame1.sample(n=50,random_state=42)

        data_frame2 = data_frame[data_frame['instability_type']!=2]
        data_frame = pd.concat([randome_instable2,data_frame2])
        self.data_frame = data_frame

        # split data into train and validation dataset
        self.train_data, self.val_data, self.train_ids, self.valid_ids = self.split_dataset()

        # pre-processing the images
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),

              #ADD MORE TRANSFORM METHODS HERE

            transforms.RandomHorizontalFlip(),## ADDDED
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)), ## ADDDED
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),## ADDDED
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])


        self.train_loader = self.create_dataloader(self.train_data, self.transform)
        self.val_loader = self.create_dataloader(self.val_data, self.transform, shuffle=False)


        # use the gpu
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # set the loss weight
        '''
        Changed some thing here : to see the results
        1. add the class_weights to each class labels with different weights,
           more lables occurece with lower weight, few label occurence with higher weight;
           without the weights, all the labels will be treated equally.

        2. used to apply L2 regularization (also called weight decay).
        The primary purpose of weight decay is to prevent overfitting by penalizing large weights.
         It adds a penalty to the loss function based on the size of the weights
         helps it generalize better to unseen data.
         Lnew = Lold + weight_decay * sum(weight^2)

        3. add scheduler to Reduces the learning rate after every step_size epochs.
          After every 4 epochs, the learning rate will be multiplied by gamma (0.1 here), reducing it by 90%.
        '''
        class_weights = torch.tensor([100/25 , 100/25, 100/20,  100/15, 100/10,  100/5], device= self.device)
        self.criterion = nn.CrossEntropyLoss(weight=class_weights) # CrossEntropy for multi categorical-label predication
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate , weight_decay=0.0001)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size = 15, gamma=0.1)


  '''
  The function helps to split the data set into the training and validation dataset according to the
  size pre-determined.
  '''

  def split_dataset(self):
    split = StratifiedShuffleSplit(n_splits=1, test_size=self.test_size, random_state=42)
    train_ids = []
    valid_ids = []
    for train_idx, val_idx in split.split(self.data_frame, self.data_frame[self.stratify_column]):
      train_data = self.data_frame.iloc[train_idx]
      val_data = self.data_frame.iloc[val_idx]
      train_ids.append(train_idx)
      valid_ids.append(val_idx)
    print(f"Train dataset size: {len(train_data)}",
       f"Validation dataset size: {len(val_data)}",
       f"length of train_ids{(len(train_ids))}",
       f"length of valid_ids{(len(valid_ids))}")
    return train_data, val_data, train_ids, valid_ids


  '''
  The function helps to loda the image
  '''
  def create_dataloader(self, data_frame, transform, shuffle=True):
    dataset = BlockStackDataset(data_frame, self.img_dir, transform=transform) # transform 可以用来数据增强
    return DataLoader(dataset, batch_size=self.batch_size, shuffle=shuffle)


  def generate_classification_report(self, outputs, labels):
    predicted = outputs # 'outputs' is already a numpy array after prediction
    labels = labels
    print(classification_report(labels, predicted, zero_division=0))



  def calculate_confusion_matrix(self, outputs, labels):
    predicted = outputs
    labels = labels
    matrix = confusion_matrix(labels, predicted)
    print(matrix)


  def validate(self):
    self.model.eval()
    val_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    all_labels = []  # add all the lables
    all_predictions = []  # add all the prediction

    with torch.no_grad():
        for inputs, labels, _ in self.val_loader:
            inputs, labels = inputs.to(self.device), labels.to(self.device).long()
            labels = labels - 1
            outputs = self.model(inputs)
            predicted = torch.argmax(outputs, 1)

            # collecting all the lables and predictions
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

            # calculate the loss
            loss = self.criterion(outputs, labels)
            val_loss += loss.item()

            # calculate the correct predication
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    # calculate the accuracy rate
    val_accuracy = correct_predictions / total_samples
    self.generate_classification_report(np.array(all_predictions), np.array(all_labels))
    self.calculate_confusion_matrix(np.array(all_predictions), np.array(all_labels))
    return val_loss/len(self.val_loader),val_accuracy, all_labels, all_predictions


  '''
  The function here is used as the main training function on the image by using the pre-definned models in
  hte first model class.
  '''

  def train(self):
      _, solution_dir = create_log_dir()

      best_val_accuracy = 0.0
      for epoch in range(self.num_epochs):
          self.model.train()
          running_loss = 0.0
          running_accuracy = 0.0

          # monitor the process
          with tqdm(self.train_loader, unit="batch") as tepoch:
              tepoch.set_description(f"Epoch {epoch + 1}/{self.num_epochs}")

              for inputs, labels, _ in tepoch:
                  inputs, labels = inputs.to(self.device), labels.to(self.device).long()
                  labels = labels - 1

                  #forward propagation
                  self.optimizer.zero_grad()
                  raw_outputs = self.model(inputs)
                  loss = self.criterion(raw_outputs, labels)  # loss calculation

                  loss.backward()  # backward propagation
                  self.optimizer.step()

                  #Loss calculating
                  running_loss += loss.item()
                  _, predicted = torch.max(raw_outputs, 1)
                  accuracy = (predicted == labels).sum().item()/ labels.size(0)
                  running_accuracy += accuracy
                  tepoch.set_postfix(loss=running_loss / len(self.train_loader),
                            accuracy=running_accuracy / len(self.train_loader))



          self.scheduler.step()
          print(self.scheduler.get_last_lr())


          val_loss,val_accuracy, all_labels, all_predictions = self.validate()
          print(f"Epoch {epoch + 1}/{self.num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

          if val_accuracy > best_val_accuracy:
              best_val_accuracy = val_accuracy
              torch.save(self.model.state_dict(), f'{solution_dir}/best_model.pth')
              print('Best model saved!')

      print('Finished Training')
      print(f'Best validation accuracy: {best_val_accuracy:.4f}')



# Model 3
* Predict total height, see the performance of instability type 0

In [10]:
# test
if __name__ == "__main__":
    model = TunnedBlockStackNet8()
    trainer = ClassificationBlockStackTrainer1(
        csv_file = './COMP90086_2024_Project_train/train.csv', ##
        img_dir='./COMP90086_2024_Project_train/train', ##
        model=model,
        test_size=0.2, # used to control the size of data in split_dataset(self)
        num_epochs=16,
        batch_size=32
        )
    trainer.train()
        

Train dataset size: 6144 Validation dataset size: 1536 length of train_ids1 length of valid_ids1


Epoch 1/16: 100%|██████████| 192/192 [05:35<00:00,  1.74s/batch, accuracy=0.647, loss=0.818]   


[0.001]
              precision    recall  f1-score   support

           1       0.79      0.99      0.88       149
           2       0.84      0.74      0.79       222
           3       0.84      0.70      0.76       304
           4       0.73      0.57      0.64       395
           5       0.74      0.94      0.83       466

    accuracy                           0.77      1536
   macro avg       0.79      0.79      0.78      1536
weighted avg       0.78      0.77      0.77      1536

[[147   2   0   0   0]
 [ 38 165  19   0   0]
 [  2  28 212  60   2]
 [  0   1  19 226 149]
 [  0   1   3  24 438]]
Epoch 1/16, Validation Loss: 0.5337, Validation Accuracy: 0.7734
Best model saved!


Epoch 2/16: 100%|██████████| 192/192 [05:33<00:00,  1.74s/batch, accuracy=0.806, loss=0.488]  


[0.001]
              precision    recall  f1-score   support

           1       0.98      0.81      0.88       149
           2       0.82      0.89      0.86       222
           3       0.88      0.78      0.82       304
           4       0.79      0.67      0.72       395
           5       0.79      0.96      0.87       466

    accuracy                           0.82      1536
   macro avg       0.85      0.82      0.83      1536
weighted avg       0.83      0.82      0.82      1536

[[120  29   0   0   0]
 [  3 198  21   0   0]
 [  0  13 236  55   0]
 [  0   0  11 265 119]
 [  0   0   1  17 448]]
Epoch 2/16, Validation Loss: 0.4094, Validation Accuracy: 0.8249
Best model saved!


...

[0.001]
              precision    recall  f1-score   support

           1       0.95      0.99      0.97       149
           2       0.91      0.95      0.93       222
           3       0.92      0.92      0.92       304
           4       0.90      0.85      0.87       395
           5       0.92      0.93      0.92       466

    accuracy                           0.92      1536
   macro avg       0.92      0.93      0.92      1536
weighted avg       0.92      0.92      0.92      1536

[[147   2   0   0   0]
 [  8 212   2   0   0]
 [  0  17 281   6   0]
 [  0   2  20 334  39]
 [  0   0   1  31 434]]
Epoch 14/16, Validation Loss: 0.2033, Validation Accuracy: 0.9167


Epoch 15/16: 100%|██████████| 192/192 [05:33<00:00,  1.73s/batch, accuracy=0.914, loss=0.227]  


[0.0001]
              precision    recall  f1-score   support

           1       0.99      0.93      0.96       149
           2       0.91      0.91      0.91       222
           3       0.92      0.87      0.89       304
           4       0.85      0.50      0.63       395
           5       0.70      0.98      0.82       466

    accuracy                           0.82      1536
   macro avg       0.87      0.84      0.84      1536
weighted avg       0.84      0.82      0.81      1536

[[138  11   0   0   0]
 [  2 202  17   1   0]
 [  0   9 264  28   3]
 [  0   0   6 198 191]
 [  0   0   1   6 459]]
Epoch 15/16, Validation Loss: 0.5960, Validation Accuracy: 0.8210


Epoch 16/16: 100%|██████████| 192/192 [05:33<00:00,  1.74s/batch, accuracy=0.937, loss=0.167]   


[0.0001]
              precision    recall  f1-score   support

           1       0.97      0.99      0.98       149
           2       0.91      0.97      0.94       222
           3       0.91      0.94      0.92       304
           4       0.92      0.89      0.91       395
           5       0.96      0.94      0.95       466

    accuracy                           0.93      1536
   macro avg       0.94      0.94      0.94      1536
weighted avg       0.94      0.93      0.93      1536

[[147   2   0   0   0]
 [  5 215   2   0   0]
 [  0  17 285   2   0]
 [  0   1  25 353  16]
 [  0   0   1  29 436]]
Epoch 16/16, Validation Loss: 0.1758, Validation Accuracy: 0.9349
Best model saved!
Finished Training
Best validation accuracy: 0.9349


In [12]:
val_loss,val_accuracy, all_labels, all_predictions = trainer.validate()

              precision    recall  f1-score   support

           1       0.96      0.99      0.97       149
           2       0.92      0.97      0.95       222
           3       0.92      0.95      0.94       304
           4       0.93      0.89      0.91       395
           5       0.96      0.94      0.95       466

    accuracy                           0.94      1536
   macro avg       0.94      0.95      0.94      1536
weighted avg       0.94      0.94      0.94      1536

[[147   2   0   0   0]
 [  6 215   1   0   0]
 [  0  15 288   1   0]
 [  0   1  22 352  20]
 [  0   0   1  26 439]]


In [14]:
df.total_height.value_counts()

total_height
6    2304
5    1920
4    1536
3    1152
2     768
Name: count, dtype: int64

In [33]:
valid_df = trainer.val_data
valid_df['pred_total_height'] = [pred+1 for pred in all_predictions]
valid_df['pred_type'] = valid_df['pred_total_height'] == valid_df['total_height']
valid_df['tt_pred_comp_stable_gt'] = valid_df['pred_total_height'] == valid_df['stable_height']
valid_df.head()

/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_29836/3244600169.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['pred_total_height'] = [pred+1 for pred in all_predictions]
/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_29836/3244600169.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['pred_type'] = valid_df['pred_total_height'] == valid_df['total_height']
/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_29836/3244600169.py:4: SettingWithCopyWarning:

,id,shapeset,type,total_height,instability_type,cam_angle,stable_height,pred_total_height,pred_type,tt_pred_comp_stable_gt
7334,956915,2,2,4,2,1,1,4,True,False
3955,516709,1,2,5,1,1,3,5,True,False
619,77447,1,2,5,1,1,3,5,True,False
1594,212770,2,2,4,2,1,1,4,True,False
5645,745098,2,2,6,1,1,4,6,True,False


In [34]:
print(" Accuracy of prediction of total height")
valid_df.groupby('instability_type')['pred_type'].sum()/valid_df.groupby('instability_type')['pred_type'].count()

 Accuracy of prediction of total height


instability_type
0    0.956640
1    0.935650
2    0.925641
Name: pred_type, dtype: float64

In [35]:
print("Accuracy of prediction of total height based on stable height")
valid_df.groupby('instability_type')['tt_pred_comp_stable_gt'].sum()/valid_df.groupby('instability_type')['tt_pred_comp_stable_gt'].count()

Accuracy of prediction of total height based on stable height


instability_type
0    0.956640
1    0.011583
2    0.010256
Name: tt_pred_comp_stable_gt, dtype: float64

In [36]:
df.head()

,id,shapeset,type,total_height,instability_type,cam_angle,stable_height
0,54,2,1,3,1,1,2
1,173,1,1,4,1,2,1
2,245,1,1,4,1,2,1
3,465,2,1,5,0,1,5
4,611,2,1,3,1,1,1


# Model 2
* add learning rate to 0.001
* Worse

In [96]:
# test
if __name__ == "__main__":
    model = TunnedBlockStackNet8()
    trainer = ClassificationBlockStackTrainer1(
        csv_file = './COMP90086_2024_Project_train/train.csv', ##
        img_dir='./COMP90086_2024_Project_train/train', ##
        model=model,
        test_size=0.2, # used to control the size of data in split_dataset(self)
        num_epochs=16,
        batch_size=32
        )
    trainer.train()
        

Train dataset size: 4648 Validation dataset size: 1162 length of train_ids1 length of valid_ids1


Epoch 1/16: 100%|██████████| 146/146 [04:09<00:00,  1.71s/batch, accuracy=0.226, loss=1.74]   


[0.01]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       259
           1       0.25      1.00      0.39       285
           2       0.00      0.00      0.00       233
           3       0.00      0.00      0.00       180
           4       0.00      0.00      0.00       128
           5       0.00      0.00      0.00        77

    accuracy                           0.25      1162
   macro avg       0.04      0.17      0.07      1162
weighted avg       0.06      0.25      0.10      1162

[[  0 259   0   0   0   0]
 [  0 285   0   0   0   0]
 [  0 233   0   0   0   0]
 [  0 180   0   0   0   0]
 [  0 128   0   0   0   0]
 [  0  77   0   0   0   0]]
Epoch 1/16, Validation Loss: 1.7159, Validation Accuracy: 0.2453
Best model saved!


Epoch 2/16: 100%|██████████| 146/146 [04:08<00:00,  1.70s/batch, accuracy=0.238, loss=1.72]  


[0.01]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       259
           1       0.25      1.00      0.39       285
           2       0.00      0.00      0.00       233
           3       0.00      0.00      0.00       180
           4       0.00      0.00      0.00       128
           5       0.00      0.00      0.00        77

    accuracy                           0.25      1162
   macro avg       0.04      0.17      0.07      1162
weighted avg       0.06      0.25      0.10      1162

[[  0 259   0   0   0   0]
 [  0 285   0   0   0   0]
 [  0 233   0   0   0   0]
 [  0 180   0   0   0   0]
 [  0 128   0   0   0   0]
 [  0  77   0   0   0   0]]
Epoch 2/16, Validation Loss: 1.7126, Validation Accuracy: 0.2453


Epoch 3/16: 100%|██████████| 146/146 [04:09<00:00,  1.71s/batch, accuracy=0.24, loss=1.72]   


[0.01]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       259
           1       0.25      1.00      0.39       285
           2       0.00      0.00      0.00       233
           3       0.00      0.00      0.00       180
           4       0.00      0.00      0.00       128
           5       0.00      0.00      0.00        77

    accuracy                           0.25      1162
   macro avg       0.04      0.17      0.07      1162
weighted avg       0.06      0.25      0.10      1162

[[  0 259   0   0   0   0]
 [  0 285   0   0   0   0]
 [  0 233   0   0   0   0]
 [  0 180   0   0   0   0]
 [  0 128   0   0   0   0]
 [  0  77   0   0   0   0]]
Epoch 3/16, Validation Loss: 1.7126, Validation Accuracy: 0.2453


Epoch 4/16:  75%|███████▍  | 109/146 [03:06<01:03,  1.71s/batch, accuracy=0.184, loss=1.28]  


KeyboardInterrupt: 

In [29]:
# test
if __name__ == "__main__":
    model = TunnedBlockStackNet8()
    trainer = ClassificationBlockStackTrainer1(
        csv_file = './COMP90086_2024_Project_train/train.csv', ##
        img_dir='./COMP90086_2024_Project_train/train', ##
        model=model,
        test_size=0.2, # used to control the size of data in split_dataset(self)
        num_epochs=16,
        batch_size=32
        )
    trainer.train()
        

Train dataset size: 4648 Validation dataset size: 1162 length of train_ids1 length of valid_ids1


Epoch 1/16: 100%|██████████| 146/146 [04:11<00:00,  1.72s/batch, accuracy=0.249, loss=1.62]  


[0.001]
              precision    recall  f1-score   support

           0       0.07      0.01      0.02       259
           1       0.50      0.22      0.31       285
           2       0.29      0.39      0.33       233
           3       0.25      0.22      0.24       180
           4       0.18      0.73      0.29       128
           5       0.00      0.00      0.00        77

    accuracy                           0.25      1162
   macro avg       0.22      0.26      0.20      1162
weighted avg       0.26      0.25      0.21      1162

[[ 3 49 72 38 97  0]
 [ 5 63 95 31 91  0]
 [ 1 10 90 41 91  0]
 [ 5  3 34 40 98  0]
 [10  0 16  8 94  0]
 [18  0  3  2 54  0]]
Epoch 1/16, Validation Loss: 1.6103, Validation Accuracy: 0.2496
Best model saved!


Epoch 2/16: 100%|██████████| 146/146 [04:13<00:00,  1.73s/batch, accuracy=0.274, loss=1.57]  


[0.001]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       259
           1       0.49      0.34      0.40       285
           2       0.30      0.45      0.36       233
           3       0.24      0.38      0.29       180
           4       0.19      0.26      0.22       128
           5       0.30      0.57      0.39        77

    accuracy                           0.30      1162
   macro avg       0.25      0.33      0.28      1162
weighted avg       0.26      0.30      0.26      1162

[[  0  70  81  52  32  24]
 [  0  96  81  55  39  14]
 [  0  26 105  60  28  14]
 [  0   2  62  68  25  23]
 [  0   0  17  49  33  29]
 [  0   2   8   5  18  44]]
Epoch 2/16, Validation Loss: 1.5226, Validation Accuracy: 0.2978
Best model saved!


...

[0.001]
              precision    recall  f1-score   support

           0       0.33      0.02      0.03       259
           1       0.33      0.85      0.48       285
           2       0.45      0.19      0.27       233
           3       0.36      0.36      0.36       180
           4       0.27      0.28      0.28       128
           5       0.50      0.05      0.09        77

    accuracy                           0.34      1162
   macro avg       0.38      0.29      0.25      1162
weighted avg       0.37      0.34      0.27      1162

[[  4 191  17  29  17   1]
 [  2 242  13  16  10   2]
 [  4 146  45  30   8   0]
 [  1  70  22  64  23   0]
 [  0  55   2  34  36   1]
 [  1  29   0   5  38   4]]
Epoch 14/16, Validation Loss: 1.5011, Validation Accuracy: 0.3399
Best model saved!


Epoch 15/16: 100%|██████████| 146/146 [04:09<00:00,  1.71s/batch, accuracy=0.365, loss=1.44]   


[0.0001]
              precision    recall  f1-score   support

           0       0.33      0.35      0.34       259
           1       0.45      0.27      0.34       285
           2       0.38      0.48      0.42       233
           3       0.26      0.48      0.34       180
           4       0.22      0.15      0.18       128
           5       0.25      0.03      0.05        77

    accuracy                           0.33      1162
   macro avg       0.31      0.29      0.28      1162
weighted avg       0.34      0.33      0.32      1162

[[ 90  51  58  47  12   1]
 [ 88  77  70  42   8   0]
 [ 20  29 112  60  11   1]
 [ 25   7  46  86  13   3]
 [ 26   7   7  68  19   1]
 [ 23   2   2  23  25   2]]
Epoch 15/16, Validation Loss: 1.5349, Validation Accuracy: 0.3322


Epoch 16/16: 100%|██████████| 146/146 [04:19<00:00,  1.78s/batch, accuracy=0.396, loss=1.39]   


[0.0001]
              precision    recall  f1-score   support

           0       0.32      0.31      0.31       259
           1       0.44      0.53      0.48       285
           2       0.51      0.36      0.43       233
           3       0.31      0.32      0.32       180
           4       0.29      0.37      0.32       128
           5       0.38      0.29      0.33        77

    accuracy                           0.38      1162
   macro avg       0.38      0.36      0.36      1162
weighted avg       0.39      0.38      0.38      1162

[[ 79 101  30  21  16  12]
 [ 58 151  25  30  17   4]
 [ 23  58  85  44  22   1]
 [ 38  19  22  58  36   7]
 [ 30   7   4  28  47  12]
 [ 19   4   0   7  25  22]]
Epoch 16/16, Validation Loss: 1.3914, Validation Accuracy: 0.3804
Best model saved!
Finished Training
Best validation accuracy: 0.3804
